In [17]:
from dask.distributed import Client
import dask.dataframe as dd
import pandas as pd
from pathlib import Path
from fsspec.registry import known_implementations
known_implementations['adl'] = {'class': 'adlfs.AzureDatalakeFileSystem'}

In [18]:
client=Client('172.20.198.209:8786')

In [19]:
client

Client Scheduler: tcp://172.20.198.209:8786 Dashboard: http://172.20.198.209:8787/status,Cluster Workers: 19 Cores: 76 Memory: 640.46 GB


In [20]:
# client.get_versions(check=True)

In [21]:
# Set the home directory
home = Path.cwd()
STORAGE_OPTIONS={'tenant_id' : 'c1eb5112-7946-4c9d-bc57-40040cfe3a91',
                 'client_secret' : 'oE63J*QVKfV*bEJN@dIQpkb2*xVr0l3_',
                 'client_id' : 'a3afc7a4-80a7-4d7c-8b27-5cc1209aff3c'}
STORE_NAME='eclwpsdatalake'

In [22]:
# Load the customer data
cust_df=pd.read_excel(Path(home, 'data', 'WH Controllers (version 1).xlsx'), sheet_name='Sheet1')
cust_df=cust_df[['ControllerID','SerialNo','Customer Name', 'City', 'State', 'Country',
       'RegionName','DATA IN LAST 3 MONTHS', 'DATA IN LAST 12 MONTHS']]
cust_df.columns=cust_df.columns.str.lower().tolist()

In [23]:
files = ['2017H1.csv', '2017H2.csv', '2018H1.csv', '2018H2.csv',
          '2019H1.csv']
cols = ['controllerid','parameterid','MeasureDate','parametername','Application','value']

#files1 = ['2018Jan1Week.csv', '2019June2Weeks.csv']
#controllers = "WH Controllers (version 1).xlsx , sheet='Sheet1'"

In [24]:
def load_file(file, storage_options, cols):
    fname = f'adl://{STORE_NAME}/WPS/Corrosion_Readings/15minsFrequency/{file}'
    ddf = dd.read_csv(fname, storage_options=storage_options, header=None)
    ddf.columns=cols
   # ddf['parametername'] = ddf['parametername'].astype('category')
   # ddf['Application'] = ddf['Application'].astype('category')
    return ddf

In [25]:
dfs = []
for file in files:
    ddf = load_file(file=file, storage_options=STORAGE_OPTIONS, cols=cols)
    dfs.append(ddf)

In [26]:
ddf2 = dd.concat(dfs, axis=0).persist()

In [27]:
ddf3 = ddf2.reset_index(drop=True)

In [28]:
ddf3._meta.dtypes

controllerid       int64
parameterid        int64
MeasureDate       object
parametername     object
Application       object
value            float64
dtype: object

In [29]:
ddf3.head()

,controllerid,parameterid,MeasureDate,parametername,Application,value
0,73468,14,2017-01-06 07:30:00.000,TRASAR Dose,CCGT- Combined Cycle,26.93
1,73468,14,2017-01-06 07:45:00.000,TRASAR Dose,CCGT- Combined Cycle,25.70
2,73468,14,2017-01-06 08:00:00.000,TRASAR Dose,CCGT- Combined Cycle,26.62
3,73468,14,2017-01-06 08:15:00.000,TRASAR Dose,CCGT- Combined Cycle,27.46
4,73468,14,2017-01-06 08:30:00.000,TRASAR Dose,CCGT- Combined Cycle,28.35


In [30]:
mem = ddf2.memory_usage(deep=True).sum().compute()//1e6
print(mem)

2163.0


In [31]:
ddf.npartitions

3

In [32]:
ddf3 = ddf2.repartition(npartitions=500).persist()

In [33]:
ddf3.head(5)

,controllerid,parameterid,MeasureDate,parametername,Application,value
0,73468,14,2017-01-06 07:30:00.000,TRASAR Dose,CCGT- Combined Cycle,26.93
1,73468,14,2017-01-06 07:45:00.000,TRASAR Dose,CCGT- Combined Cycle,25.70
2,73468,14,2017-01-06 08:00:00.000,TRASAR Dose,CCGT- Combined Cycle,26.62
3,73468,14,2017-01-06 08:15:00.000,TRASAR Dose,CCGT- Combined Cycle,27.46
4,73468,14,2017-01-06 08:30:00.000,TRASAR Dose,CCGT- Combined Cycle,28.35


In [35]:
outfile = f'adl://{STORE_NAME}/WPS/EDA/Corrosion_Readings/readings15mins.parquet'

In [36]:
dd.to_parquet(ddf3, outfile, storage_options=STORAGE_OPTIONS, engine='pyarrow')

In [37]:
client.close()